In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle
import plotly as py
from plotly.offline import plot, iplot
from scipy import stats
import plotly.graph_objs as go
from sklearn.cluster import DBSCAN
py.offline.init_notebook_mode(connected=True)

%matplotlib notebook

In [ ]:
# path in the file
path_to_data = "/home/samba693/DataChallenge/debs2019_dataset2/"
path_to_outliers = "/home/samba693/Documents/DEBS/debs2019/src/ssh-kd/data/outliers.pkl"

In [ ]:
# Read in the file
df = pd.read_csv(path_to_data+"in.csv",usecols=[1,2,3,4],names=["laser_id","X","Y","Z"])
num_of_scenes = len(df) / 72000
dataframes = []

for i in range(int(num_of_scenes)):
    start = i * 72000
    end = start + 72000
    dataframes.append(df.iloc[start:end, :])

In [ ]:

out_file = open(path_to_data+"out.csv", 'r').readlines()
outfile = [i.rstrip() for i in out_file]
outfile = [i.split(',') for i in outfile]
outfile = [i[1:] for i in outfile]

outfile_list = []
out_cluster = []
for i in outfile:
    a  = {}
    for j in range(0,len(i),2):
        a[i[j]] = int(i[j+1])
    out_cluster.append(sum(a.values()))
    outfile_list.append(a)

In [ ]:
# remove the outliers
def remove_out(df,path_out = path_to_outliers):
    pd.options.mode.chained_assignment = None
    outliers = pd.read_pickle(path_out)
    max_rad = outliers[0]
    min_rad = outliers[1]

    df["radius"] = df.X.pow(2).add(df.Y.pow(2).add(df.Z.pow(2))).pow(0.5).round(1)
    rad = np.array(df.radius)
    bool_vec = (rad <= max_rad) & (rad >= min_rad)
#     bool_vec = (rad >= min_rad)
    df = df[~bool_vec]
    df.drop(df[df["radius"] == 0].index, inplace=True)
#     df = df[df['laser_id']>30]
    
    return df

In [ ]:
def plot_3d(df):
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    ax.scatter(df['X'],df['Z'],df['Y'], marker='o')

In [ ]:
def plot_plot(object_data_frame,labels=False,save=False):
    if labels:
        labels1 = tuple(object_data_frame['labels'].tolist())
        use_labels = labels1
    else:
        use_labels = 'black'
    x = tuple(object_data_frame['X'].tolist())
    y = tuple(object_data_frame['Y'].tolist())
    z = tuple(object_data_frame['Z'].tolist())
    
    trace = go.Scatter3d(
            x = x,
            y = z,
            z = y,
            mode = 'markers',
            marker = dict(
                        color=use_labels,
                        colorscale='Viridis',
                        size = 1,
                        opacity = 0.8)
            )
    layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=0, range = [-150,150],),
                    yaxis = dict(
                        nticks=0, range = [-100,100],),
                    zaxis = dict(
                        nticks=0, range = [-10,100],),),
                  )
    data=[trace]
    fig = go.Figure(data=data, layout=layout)
    if save:
        plot(fig, filename = "scene_0_{}.html".format(labels1[0]),auto_open=False, show_link = False)
    else:
        iplot(fig)

In [ ]:
# used for plots for paper

# temp = dataframes[0]
# temp = temp[(temp['X']>=0) & (temp['Y']<=20) & (temp['Z']>=0)]
# plot_plot(temp)

# temp1 = remove_out(dataframes[0])
# temp1 = temp1[(temp1['X']>=0) & (temp1['Y']<=20) & (temp1['Z']>=0)]
# plot_plot(temp1)

# # clustering with DBSCAN and removing noise
# def doClusteringWithDBSCAN(data):    
#     clustering = DBSCAN(eps=1, min_samples=16).fit(data)
#     labels = clustering.labels_
#     return labels
# data =  np.array(
#                 list(
#                     zip(
#                         np.array(temp1['X']), 
#                         np.array(temp1['Y']), 
#                         np.array(temp1['Z'])
#                     )
#                 )
#             )

# labels = doClusteringWithDBSCAN(data)

# temp1['labels'] = labels

# temp2 = temp1[~(temp1['labels']==-1)]

# plot_plot(temp2,True)


In [ ]:
def removeOutWithDBSCAN(temp1, top=False, count=0, objects=False):
    if top:
        data =  np.array(
                list(
                    zip(
                        np.array(temp1['X']), 
                        np.array(temp1['Z'])
                    )
                )
            )
    else:
        data =  np.array(
                    list(
                        zip(
                            np.array(temp1['X']), 
                            np.array(temp1['Y']), 
                            np.array(temp1['Z'])
                        )
                    )
                )
        
    clustering = DBSCAN(eps=1, min_samples=16).fit(data)
    labels = clustering.labels_
    temp1['labels'] = labels
    temp1 = temp1[~(labels==-1)]
    if objects:
        temp1['labels'] = temp1['labels']+count
    return temp1

In [ ]:
import math
# calculating degree
calDegrees=lambda x : round(math.degrees(math.atan(x)), 1)
vfunc = np.vectorize(calDegrees)
def tang(x,y):
    """ Get Tan inverse of y/x to get the angle."""
    tag= np.divide(y, x)
    return tag


def get_degrees(c):
    """ Get the degree for the data based upon cordinate plane.
    Tan behaviour in cordinate system.
    1st cord: theta
    2nd cord: 180+theta
    3rd cord: 180+theta
    4th cord: 360+theta
    return: degree of dataframe rows
    """
    
    if c['X']==0 and c['Z']>0:
        return 90
    if c['X']==0 and c['Z']<0:
        return 270
    
    degrees =  vfunc(tang(c['X'],c['Z']))
    if c['X']<0:
        degrees = 180+degrees
    if (c['X']>0 and c['Z']<0):
        degrees = 360+degrees
        
    return degrees

def angle_of_elevation(x,y,z):
    """Get Tan inverse of y/sqrt(x^2+z^2) to get the angle"""
    den = math.sqrt(x**2+z**2)
    etan= np.divide(y,den)
    return etan 


def get_phi(c):
    """ Get the degree for the data based upon cordinate plane.
    return: degree of dataframe rows
    """
    phi =  vfunc(angle_of_elevation(c['X'],c['Y'],c['Z']))
    phi =  90-phi
    return np.float(phi)

def get_r(x,y,z):
    """Get density r"""
    den = math.sqrt(x**2+z**2)
    return den 

def get_den(c):
    r = get_r(c['X'],c['Y'],c['Z'])
    return r

In [ ]:
# Using Dataframe apply, a func is run on dataframe, pass the datafram to get_degrees

def get_different_sectors(temp, threshold=0.7):
    temp['angles'] = temp.apply(get_degrees,axis=1)
    temp['top_rad'] = temp.apply(get_den,axis=1)
    temp.angles = temp.angles.astype(np.float)
    unique_angles = sorted(list(set(temp.angles.value_counts().index)))
    indexes_to_split = list(np.where(np.diff(unique_angles)>=threshold)[0]+1)
    start=0
    if len(unique_angles)==0:
        angle_ranges = [unique_angles]
    else:
        angle_ranges = []
    for i in indexes_to_split:
        angle_ranges.append(unique_angles[start:i])
        start = i
    list_of_sectors = []
    angles = np.array(temp.angles)
    for j in angle_ranges:
        max_angle = max(j)
        min_angle = min(j)
        bool_vec = (angles>=min_angle) & (angles<=max_angle)
        list_of_sectors.append(temp[bool_vec])
    list_of_valid = []
    for i,j in enumerate(list_of_sectors):
        if len(j)>10:
            j['labels'] = i
            list_of_valid.append(j)
    return list_of_valid

def get_valid_density(list_of_valid_sec):
    list_of_valid_density = []
    for temp in list_of_valid_sec:
        # Initializing the kernel
        kernel = stats.gaussian_kde(temp['top_rad'].to_numpy(),bw_method=0.05)

        # Evaluating the values
        to_plot2 = kernel.evaluate(np.linspace(-20,180,500))
        # plt.plot(np.linspace(-20,180,500),to_plot2)
        # Threshold ==0.01
        bool_vec = [~(to_plot2<=0.001)]

        # Selecting valid values wrt threshold
        to_plot = to_plot2[bool_vec]
        x_val = np.linspace(-20,180,500)[bool_vec]

        # Selecting the boundary points
        req_indexes = np.where((np.diff(x_val)<=0.5)==False)[0]+1
        markers_on = x_val[req_indexes].round(0).astype(int).tolist()
        markers_on = [0]+markers_on+[180]

        # Calculate the dense indexs
        to_dense = np.split(to_plot,req_indexes)
        try:
            max_dense = [np.max(j) for j in to_dense]
        except:
#             print(x_val)
#             print(to_dense)
#             print(req_indexes)
            list_of_valid_density.append(temp)

    #     print(markers_on)
    #     print(max_dense)

        # Selecting the valid objects
        for i in range(len(markers_on)-1):
            if max_dense[i]>=0.01:
                temp1 = temp[ (temp['top_rad']>=markers_on[i]) & (temp['top_rad']<=markers_on[i+1]) ]
                if len(temp1)>=15:
                    list_of_valid_density.append(temp1)
    return list_of_valid_density

In [ ]:
# plot the different sectors
def plot_sectors(list_of_sec,labels = True, save=False):
    df = pd.concat(list_of_sec)
    plot_plot(df, labels, save)

In [ ]:
# Radius density
# temp = list_of_valid_sec[13]
# temp['top_rad'] = (temp['top_rad']**0.5).round(1)
# temp['top_rad'].plot.kde(bw_method=0.1,figsize=(10,10))

In [ ]:
# # plot_sectors([list_of_valid_density[2]],True)
# # plot_sectors(list_of_valid_density,True)
# # plot_sectors([list_of_valid_sec[2]],True)

# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# count=0
# for i in dataframes:
#     list_of_valid_sec = get_different_sectors(remove_out(i))
#     list_of_valid_density = get_valid_density(list_of_valid_sec)
#     print("original clusters = {}, number of detected = {}".format(out_cluster[count],len(list_of_valid_density)))
#     count+=1

# list_of_valid_sec = get_different_sectors(remove_out(dataframes[0]))
# list_of_valid_density = get_valid_density(list_of_valid_sec)
# print("original clusters = {}, number of detected = {}".format(out_cluster[0],len(list_of_valid_density)))

In [ ]:
# for i in range(len(markers_on)-1):    
#     basic_plot3 = basic_plot2[ (basic_plot2['X']>=markers_on[i]) & (basic_plot2['X']<=markers_on[i+1])]
#     plt.plot(basic_plot3['X'].to_numpy(),basic_plot3['Y'].to_numpy())

# out_cluster[3]
# plot_sectors(list_of_valid_density,True)

In [ ]:
# for i in range(len(markers_on)-1):    
#     temp1 = temp[ (temp['top_rad']>=markers_on[i]) & (temp['top_rad']<=markers_on[i+1]) ]
#     if len(temp1)>=20:
#         list_of_valid_density.append(temp1)
# plot_plot(remove_out(dataframes[499]))

In [ ]:
# plot_sectors([list_of_valid_density[11]],True)

In [ ]:
# plot_sectors([list_of_valid_sec[13]], True)
# print(out_cluster[3])
# print(outfile_list[3])
# print(len(list_of_valid_sec))

In [ ]:
# # return the unique phi and angle
# def get_phi_sectors(list_of_valid_sec,thresold = 0.6):
#     list_of_sec_phi = []
#     for ij in list_of_valid_sec:
#         ij['phi'] = ij.apply(get_phi,axis=1)
#         unique_phi = sorted(list(set(ij.phi.value_counts().index)))
#         indexes_to_split = list(np.where(np.diff(unique_phi)>=thresold)[0]+1)
#         start=0
#         if len(indexes_to_split)==0:
#             phi_ranges = [unique_phi]
#         else:
#             phi_ranges = []
#         for i in indexes_to_split:
#             phi_ranges.append(unique_phi[start:i])
#             start = i

#         angles = np.array(ij.phi)
#         for j in phi_ranges:
#             max_angle = max(j)
#             min_angle = min(j)
#             bool_vec = (angles>=min_angle) & (angles<=max_angle)
#             list_of_sec_phi.append(ij[bool_vec])

#     list_of_valid_sec_phi = []
#     for i,j in enumerate(list_of_sec_phi):
#         if len(j)>10:
#             j['labels'] = i
#             list_of_valid_sec_phi.append(j)
#     return list_of_valid_sec_phi

In [ ]:
# list_of_valid_sec_phi = get_phi_sectors(list_of_valid_sec)

In [ ]:
# plot_sectors([list_of_valid_sec_phi[13]],True)
# print(len(list_of_valid_sec_phi))

In [ ]:
# # return the unique phi and angle
# def get_valid_r(list_of_sec_phi, thresold=2):
#     list_of_r = []
#     for ij in list_of_valid_sec_phi:
#         ij['r'] = ij.apply(get_den,axis=1)
#         temp = ij.r.round(1).value_counts()
#         unique_r = sorted(list(set(temp[temp>5].index)))
#         indexes_to_split = list(np.where(np.diff(unique_r)>=thresold)[0]+1)
#         start=0
#         if len(indexes_to_split)==0:
#             phi_ranges = [unique_r]
#         else:
#             phi_ranges = []
#         for i in indexes_to_split:
#             phi_ranges.append(unique_r[start:i])
#             start = i

#         angles = np.array(ij.r)
#         for j in phi_ranges:
#             if len(j)!=0:
# #                 print(j)
#                 max_angle = max(j)
#                 min_angle = min(j)
#                 bool_vec = (angles>=min_angle) & (angles<=max_angle)
#                 list_of_r.append(ij[bool_vec])

#     list_of_valid_r = []
#     for i,j in enumerate(list_of_r):
#         if len(j)>10:
#             j['labels'] = i
#             list_of_valid_r.append(j)
#     return list_of_valid_r

In [ ]:
# list_of_valid_r = get_valid_r(list_of_valid_sec_phi)

In [ ]:
# plot_sectors(list_of_valid_r)
# # plot_sectors([list_of_valid_r[0]])
# print(len(list_of_valid_r[0]))

In [ ]:
# # segmenting intot clusters
# def segment(df):
#     objects = []
#     for i,j in enumerate(df):
#         temp = get_valid_r(get_phi_sectors(get_different_sectors(remove_out(j))))
#         objects.append(temp)
#         print("original clusters = {}, number of detected = {}".format(out_cluster[i],len(temp)))
#     return objects

In [ ]:
# list_of_objects = segment(dataframes)

In [ ]:
# list_of_valid_sec[10]

In [ ]:
# z = list_of_valid_sec[9]['Z'].to_numpy()
# x = list_of_valid_sec[9]['X'].to_numpy()

In [ ]:
# to_check = np.round(np.arctan2(z,x)*180/np.pi,1)
# to_check[to_check<0] = to_check[to_check<0]+360

In [ ]:
# original = list_of_valid_sec[9]['angles'].to_numpy()
# to_check==original

In [ ]:
# original

In [ ]:
# to_check

In [ ]:
# np.unique(to_check)

In [ ]:
# np.sum(to_check==220.7)

In [ ]:
# temppppp = np.hstack([original.reshape(-1,1),to_check.reshape(-1,1),original.reshape(-1,1)])

In [ ]:
# np.hstack([temppppp,original.reshape(-1,1),to_check.reshape(-1,1)])

In [ ]:
"""
    - First Remove the ground points
    - Divide the points into sectors
    - Segment the objects according to the density
    - Use XZ DBSCAN to segment these objects
    - Plot the results and investigate
"""
temp = dataframes[0]
no_out = remove_out(temp)
sectors = get_different_sectors(no_out)
densities = get_valid_density(sectors)
objects = []
count = 0

for each in densities:
    temp1 = removeOutWithDBSCAN(each, True,count, True)
    if len(temp1)>=1:
#         print(temp1)
        count+=len(temp1['labels'].value_counts())
        objects += [ pd.DataFrame(y) for x,y in temp1.groupby('labels',as_index=False) ]
    

In [ ]:
# print("Predicted objects",len(objects))
# print("Densities",len(densities))
# print(out_cluster[0])
# outfile_list[0]

In [ ]:
# for i in objects:
#     plot_sectors([i],True, True)

In [ ]:
# plot_sectors(objects,True)